In [2]:
import torch
import torchaudio
import torch.nn as nn
from tqdm import tqdm
from difflib import SequenceMatcher
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, DataLoader
import time
from librosa.core import stft, magphase
from torch.autograd import Variable
from data_load import CodeSwitchDataset, TextTransform
import csv 
import copy
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import librosa
import pandas as pd 
import cv2
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler 
import numpy as np

/home/dexter/Desktop/Projects/CodeSwitching/hparam.py:11: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  for doc in docs:


In [1]:
%%time
import os 
#drive.mount('/content/drive')
from glob import glob
import syft as sy
import torchaudio
from sklearn.model_selection import train_test_split
from syft.frameworks.torch.fl import utils
from syft.workers.websocket_client import WebsocketClientWorker

CPU times: user 4.21 s, sys: 799 ms, total: 5 s
Wall time: 20.9 s


In [3]:
train_audio_transforms = nn.Sequential(
            torchaudio.transforms.MelSpectrogram(n_mels=128,
                                                 sample_rate=22050,
                                                 n_fft = 512,
                                                 win_length=int(22050*0.02),
                                                 hop_length=int(22050*0.01)),
            torchaudio.transforms.FrequencyMasking(freq_mask_param=15),
            torchaudio.transforms.TimeMasking(time_mask_param=35)
        )


valid_audio_transforms = torchaudio.transforms.MelSpectrogram()

text_transform = TextTransform()


def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, utterance, _) in data:
        waveform=torch.Tensor(waveform)
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, padding_value = 1, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths

In [4]:
def makeNonFedData():
  dcm_path=os.listdir('/content/drive/My Drive/Fed_Covid/minibatch/')
  dcm_data={}
  alpha = 1.5 
  beta = 0
  labels=[]
  pid=[]
  dicom=[]
  label=[]
  for file in dcm_path:
    name = '/content/drive/My Drive/Fed_Covid/minibatch/' + file
    temp = pydicom.dcmread(name)
    image = temp.pixel_array
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    adjusted = cv2.resize(image,(128,128))
    dcm_data[file]=adjusted  
  with open('/content/drive/My Drive/Fed_Covid/stage_2_train_labels.csv','r') as file:
    reader = csv.reader(file)
    for row in reader:
      labels.append(row)  
  scaler = MinMaxScaler()
  cid = 0
  for PID in labels:
    for key in dcm_data:
      if(key[:-4]==PID[0]):
        l=[]
        for val in dcm_data[key]:
          l.append(scaler.fit_transform(val))
        l = np.reshape(l,(3,128,128))
        dicom.append(l)
        label.append(int(PID[5]))
  
  return dicom,label


def makeNonFedData(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, utterance, _) in data:
        waveform=torch.Tensor(waveform)
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, padding_value = 1, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths

In [5]:
dicom, label = makeNonFedData()

TypeError: makeNonFedData() missing 1 required positional argument: 'data'

In [6]:
class Arguments():
    def __init__(self):
        self.batch_size = 4
        self.test_batch_size = 4
        self.epochs = 60
        self.lr = 5e-4
args = Arguments()
hparams = {
        "n_cnn_layers": 4,
        "n_rnn_layers": 5,
        "rnn_dim": 512,
        "n_class": 4,
        "n_feats": 128,
        "stride": 2,
        "dropout": 0.1,
        "learning_rate": 5e-4,
        "batch_size": 4,
        "epochs": 60
    }

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dicom,label,test_size=0.3)
test_df = pd.DataFrame()
test_df['features']=x_test
test_df['labels']=y_test
x_train= np.array(x_train)
y_train= np.array(y_train)
x_test= np.array(x_test)
y_test= np.array(y_test)

In [7]:
class CodeSwitchDataset(Dataset):
    def __init__(self, lang, mode = "train", shuffle=True):
        self.mode = mode
        # data path
        self.lang = lang
        if self.lang == "Gujarati":
            self.max_len = 0
        elif self.lang == "Telugu":
            self.max_len = 529862
        elif self.lang == 'Tamil':
            self.max_len = 0
        else:
            raise Exception("Check Language")
        if self.mode == "train":
            self.path = 'Data/PartB_{}/Train/'.format(self.lang)
        elif self.mode == "test":
            self.path = self.path = 'Data/PartB_{}/Dev/'.format(self.lang)
        else:
            raise Exception("Incorrect mode")
        self.file_list = os.listdir(os.path.join(self.path, 'Audio'))
        self.shuffle=shuffle
        self.csv_file = pd.read_csv(self.path + 'Transcription_LT_Sequence_Frame_Level_200_actual.tsv', header=None, sep='\t')
        self.input_length = []
        self.label_length = []

    def __len__(self):
        return len(self.csv_file)

    def pad(self, wav, trans, max_len):
        orig_lens = [len(wav), len(trans)]
        while len(wav) < max_len:
            diff = max_len - len(wav)
            ext = wav[:diff]
            ext = [0]*diff
            wav = np.append(wav, ext)
            ratio = int(len(trans)*diff/len(wav))
            trans_append = ''.join(['0']*ratio)
            trans +=trans_append
        return wav, trans, orig_lens

    def preprocess(self, wav, sr, trans):

        out = stft(wav, win_length=int(sr*0.02), hop_length=int(sr*0.01))
        text_transform = TextTransform()
        trans = torch.Tensor(text_transform.text_to_int(trans.lower()))

        out = magphase(out)[0]
        out = [np.log(1 + x) for x in out]
        return np.array(out), trans

    def __getitem__(self, idx):
        file_name = self.csv_file[0][idx]
        trans = self.csv_file[1][idx]
        wav, sr = librosa.load(glob(self.path + 'Audio/*'+ str(file_name) + '.wav')[0])
        
        wav, trans, lens = self.pad(wav, trans, self.max_len)
        if len(set(trans)) > 2:
            label = 1
        elif len(set(trans)) == 1 or len(set(trans)) == 2:
            label = 0
        else:
            raise Exception("Check transcript")
        if self.mode =="train":
            return wav, sr, trans, self.lang, lens
        elif self.mode == "test":
            return wav
        else:
            raise Exception("Incorrect Mode")

In [8]:
train_dataset = CodeSwitchDataset(lang='Telugu', mode="train")
validation_split = 0.2
shuffle_dataset = True
random_seed = 42
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [9]:
class data_maker(Dataset):
    def __init__(self, images, labels):
        self.data = images 
        self.targets = labels 
        self.to_torchtensor()

    def __len__(self):
        return len(self.data)

    def to_torchtensor(self):      
        self.data=torch.from_numpy(self.data)
        self.labels=torch.from_numpy(self.targets)

    def __getitem__(self, idx):
        sample=self.data[idx]
        target=self.targets[idx]
        return sample,target

In [10]:
hook = sy.TorchHook(torch)
client1 = sy.VirtualWorker(hook, id="client1")
client2 = sy.VirtualWorker(hook, id="client2")

In [11]:
%%time
#federated_train_data = data_maker(x_train,y_train).federate((hospital,clinic))
federated_train_data = train_dataset.federate((client1, client2))

KeyboardInterrupt: 

In [8]:
# federated_train_loader = sy.FederatedDataLoader(federated_train_data,batch_size=args.batch_size)
BATCH_SIZE = 4
federated_train_loader = sy.FederatedDataLoader(federated_train_dataset,
                                                batch_size=BATCH_SIZE,
                                                drop_last=True,
                                                num_workers = 6,
                                                sampler = train_sampler,
                                                collate_fn = lambda x: data_processing(x, 'train'))
#test_data = data_maker(x_test,y_test)
#test_loader = torch.utils.data.DataLoader(test_data, batch_size=args.test_batch_size)

NameError: name 'train_dataset' is not defined

In [17]:
test_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          drop_last=True,
                          num_workers=6,
                          sampler=valid_sampler,
                         collate_fn=lambda x: data_processing(x, 'valid'))

In [ ]:
class CNNLayerNorm(nn.Module):
    """Layer normalization built for cnns input"""
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time) 


class ResidualCNN(nn.Module):
    """Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
        except with layer norm instead of batch norm
    """
    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResidualCNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x # (batch, channel, feature, time)


class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x


class SpeechRecognitionModel(nn.Module):
    """Speech Recognition Model Inspired by DeepSpeech 2"""

    def __init__(self, n_cnn_layers, n_rnn_layers, rnn_dim, n_class, n_feats, stride=2, dropout=0.1):
        super(SpeechRecognitionModel, self).__init__()
        n_feats = n_feats//2
        self.cnn = nn.Conv2d(1, 64, 3, stride=stride, padding=3//2)  # cnn for extracting heirachal features

        # n residual cnn layers with filter size of 32
        self.rescnn_layers = nn.Sequential(*[

            ResidualCNN(64, 64, kernel=3, stride=1, dropout=dropout, n_feats=n_feats) 
            for _ in range(n_cnn_layers)
        ])
        self.fully_connected = nn.Linear(n_feats*64, rnn_dim)
        self.birnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=i==0)
            for i in range(n_rnn_layers)
        ])
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim*2, rnn_dim),  # birnn returns rnn_dim*2
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2) # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.birnn_layers(x)
        x = self.classifier(x)
        return x

In [ ]:
client1_model = SpeechRecognitionModel().double()
client2_model = SpeechRecognitionModel().double()
client1_optimizer = optim.Adam(client1_model.parameters(), lr=0.0001)
client2_optimizer = optim.Adam(client2_model.parameters(), lr=0.0001)
models = [hospital_model, client2_model]
optimizers = [client1, client2_optimizer]
model = SpeechRecognitionModel().double()

In [ ]:
compute_nodes = [client1, client2]

In [ ]:
def update(data, target, model, optimizer):
    loss = nn.CrossEntropyLoss()
    model.send(data.location) 
    optimizer.zero_grad()
    output = model(data)
    out = loss(output, target)
    out.backward()
    optimizer.step()
    return model

In [ ]:
def train():
  for batch_idx, (data, target) in enumerate(federated_train_loader):
    for index in range(len(compute_nodes)):
      models[index] = update(data,target,models[index],optimizers[index]) 
    for model in models:
      model.get()
    return utils.federated_avg({"hospital": models[0],"clinic":models[1]}) #


In [ ]:
# for i,val in enumerate(federated_train_loader):
#   print(val[0][0][0][0][0])

In [ ]:
# for i,val in enumerate(test_loader):
#   print(val[0][0][0][0][0])

In [ ]:
def test(federated_model):
  federated_model.eval()
  test_loss = 0
  correct = 0
  loss = nn.CrossEntropyLoss()
  with torch.no_grad():
    for data, target in test_loader:
          output = model(data)
          out = loss(output,target)
          test_loss += out
          pred = output.argmax(1, keepdim=True)
          correct += pred.eq(target.view_as(pred))

  test_loss /= len(test_loader.dataset)
  print(correct)
  print('For Global Model Test loss:' + str(test_loss))

In [ ]:
for epoch in range(5):
  start_time = time.time()
  print(f"Epoch Number {epoch}")
  federated_model = train()
  total_time = time.time() - start_time 
  print('Comm time', round(total_time,2),'s\n')

Epoch Number 0
Loss for local iteration (Wrapper)>[PointerTensor | me:78526597205 -> hospital:68451090450]
Loss for local iteration (Wrapper)>[PointerTensor | me:93920035238 -> hospital:94201163823]
Comm time 0.72 s

Epoch Number 1
Loss for local iteration (Wrapper)>[PointerTensor | me:12954878799 -> hospital:4110708716]
Loss for local iteration (Wrapper)>[PointerTensor | me:22337327273 -> hospital:33106492512]
Comm time 0.7 s

Epoch Number 2
Loss for local iteration (Wrapper)>[PointerTensor | me:35929974157 -> hospital:91467735734]
Loss for local iteration (Wrapper)>[PointerTensor | me:71801368172 -> hospital:4244658442]
Comm time 0.71 s

Epoch Number 3
Loss for local iteration (Wrapper)>[PointerTensor | me:17995056656 -> hospital:31687279716]
Loss for local iteration (Wrapper)>[PointerTensor | me:36327091581 -> hospital:37669575752]
Comm time 0.71 s

Epoch Number 4
Loss for local iteration (Wrapper)>[PointerTensor | me:34091768896 -> hospital:2876199771]
Loss for local iteration (Wra

In [ ]:
test(federated_model)

Test loss of the iteration on local model -  tensor(0.6586, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(1.3163, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(2.0114, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(2.7073, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(3.3291, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(3.9482, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(4.6063, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(5.2640, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(5.9226, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(6.6930, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(7.3148, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(7.9383, dtype=torch.float64)
Test loss of the iteration o

In [21]:
5e-4

0.0005